## Importing training data:

In [56]:
import pandas as pd

train_path = '/kaggle/input/competitions/playground-series-s6e2/train.csv'
train_df = pd.read_csv(train_path)
train_df.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [57]:
target = 'Heart Disease' 

#Encoding the target
train_df[target] = train_df[target].map({'Absence':0, 'Presence':1})

#removing redundant features and seperating features and target
X = train_df.drop(columns=['id',target], axis=1)
y = train_df[target]

## Feature engineering:

### Binning transformer:

In [58]:
from sklearn.base import BaseEstimator, TransformerMixin

class Binning(BaseEstimator, TransformerMixin):
    def __init__(self, col_to_bin, num_bins, new_col_name ,labels=None):
        self.col_to_bin = col_to_bin
        self.num_bins = num_bins
        self.labels = labels
        self.new_col_name = new_col_name

    def fit(self, X, y=None):
        X = X.copy()
        _, self.bin_edges = pd.cut(X[self.col_to_bin], bins=self.num_bins, labels=False, retbins=True) #the last attribute ensures that the same bins are used to bin teh test data during transform
        return self

    def transform(self,X):
        X = X.copy() 
        X[self.new_col_name] = pd.cut(X[self.col_to_bin], bins=self.bin_edges, labels=False)
        return X

### GroupMean Encoder:

In [59]:
class GroupMeanEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, groupby_col, agg_col, new_col_name):
        self.groupby_col = groupby_col
        self.agg_col = agg_col
        self.new_col_name = new_col_name

    def fit(self,X,y=None):
        self.means = X.groupby(self.groupby_col,observed=True)[self.agg_col].mean()
        return self

    def transform(self,X):
        X = X.copy()
        X[self.new_col_name] = X[self.groupby_col].map(self.means)
        return X

### Frequency Encoder: 

In [60]:
class FreqEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols, normalize=True):
        self.cat_cols = cat_cols
        self.normalize = normalize
        self.freq_maps = {}

    def fit(self, X, y=None):
        for col in self.cat_cols:
            self.freq_maps[col] = X[col].value_counts(normalize=self.normalize)
        return self

    def transform(self, X):
        X = X.copy()

        for col in self.cat_cols:
            X[col + '_freq'] = X[col].map(self.freq_maps[col])
            X[col + '_freq'] = X[col + '_freq'].fillna(0) 
        return X

### Target Encoder:

In [61]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols):
        self.cat_cols = cat_cols
        self.mean_maps = {}
        self.global_mean = None
        
    def fit(self, X, y):
        X = X.copy()
        self.global_mean = y.mean()

        for col in self.cat_cols:
            self.mean_maps[col] = y.groupby(X[col]).mean()
        return self

    def transform(self, X):
        X = X.copy()
        
        for col in self.cat_cols:
            X[col + "_TE"] = X[col].map(self.mean_maps[col])
            X[col + "_TE"] = X[col + "_TE"].fillna(self.global_mean) #handling unseen values
        return X

### Feature Clustering:
Added this to teh preprocessing pipeline, might serve as a good feature

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

class Clustering(BaseEstimator, TransformerMixin):
    def __init__(self, num_clusters=4, add_distance=True ,random_state=42):
        self.num_clusters = num_clusters
        self.random_state = random_state
        self.add_distance = add_distance

    def fit(self, X, y=None):
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X)

        self.kmeans = KMeans(
            n_clusters = self.num_clusters,
            random_state = self.random_state,
            n_init = 10
        )

        self.kmeans.fit(X_scaled)
        return self

    def transform(self, X):
        X = X.copy()
        X_scaled = self.scaler.transform(X)
        cluster_labels = self.kmeans.predict(X_scaled)
        X["Cluster"] = cluster_labels

        if self.add_distance: #i.e. if add_distance==True
            distances = self.kmeans.transform(X_scaled) #distance of the point from each cluster center
            
            for i in range(self.num_clusters):
                X[f"Clust_dist_{i}"] = distances[:,i] #1 column for distance from each cluster
                
        return X

## Preprocessing pipelines:

In [63]:
cat_cols = ['Sex','Chest pain type','FBS over 120','Exercise angina','EKG results']

In [64]:
from sklearn.pipeline import Pipeline

preprocessor = Pipeline([
    ('Binning', Binning(col_to_bin='Age', num_bins=3, new_col_name='Age_bins')),
    ('GroupMeanEncoder_BP', GroupMeanEncoder(groupby_col='Age_bins', agg_col='BP', new_col_name='X1')),
    ('GroupMeanEncoder_Cholesterol', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Cholesterol', new_col_name='X2')),
    ('GroupMeanEncoder_HR', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Max HR', new_col_name='X3')),
    ('FreqEncoding', FreqEncoder(cat_cols=cat_cols)),
    ('TargetEncoding', TargetEncoder(cat_cols=cat_cols)),
    ('Clustering', Clustering(num_clusters=3, add_distance=False))
])

## Optuna Tuning:

### XGBoost:

In [65]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):

    preprocessor = Pipeline([
        ('Binning', Binning(col_to_bin='Age', num_bins=3, new_col_name='Age_bins')),
        ('GroupMeanEncoder_BP', GroupMeanEncoder(groupby_col='Age_bins', agg_col='BP', new_col_name='X1')),
        ('GroupMeanEncoder_Cholesterol', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Cholesterol', new_col_name='X2')),
        ('GroupMeanEncoder_HR', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Max HR', new_col_name='X3')),
        ('FreqEncoding', FreqEncoder(cat_cols=cat_cols)),
        ('TargetEncoding', TargetEncoder(cat_cols=cat_cols)),
        ('Clustering', Clustering(num_clusters= trial.suggest_int("num_clusters",3,8), add_distance=True))
    ])

    params = {
        # Core learning parameters
        "n_estimators": trial.suggest_int("n_estimators", 500, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        
        # Tree complexity control
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        
        # Sampling 
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
        
        # Regularization 
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),

        # Performance
        "random_state": 42,
        "eval_metric": "logloss",
        "tree_method": "hist",   
        "verbosity": 0,
        "device":'cuda'
    }

    model = Pipeline([
        ('prep', preprocessor),
        ('XGB', XGBClassifier(**params))
    ])

    score = cross_val_score(model, X, y, cv=5, scoring="roc_auc").mean()

    return score

In [66]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=75)

In [67]:
# print(study.best_params)
# print(study.best_value)

Results of tuning:
* Best hyperparameter set: {'n_estimators': 2634, 'learning_rate': 0.0468112533159457, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 1.4373368015867447, 'subsample': 0.7758150044556165, 'colsample_bytree': 0.39762653718463103, 'reg_alpha': 0.1260400161073486, 'reg_lambda': 0.00014786267797526253}
* Best score: 0.9555482973765608

In [68]:
best_params_xgb = {'n_estimators': 2634, 
                   'learning_rate': 0.0468112533159457, 
                   'max_depth': 3, 
                   'min_child_weight': 5, 
                   'gamma': 1.4373368015867447, 
                   'subsample':  0.7758150044556165, 
                   'colsample_bytree': 0.39762653718463103, 
                   'reg_alpha': 0.1260400161073486, 
                   'reg_lambda': 0.00014786267797526253,
                  
                    'random_state':42,
                    'eval_metric':"logloss",
                    'tree_method':"hist",
                    'device':"cuda",
                    'verbosity':0}

## Retraining the model:

In [69]:
best_xgb_model = Pipeline([
        ('prep', preprocessor),
        ('XGB', XGBClassifier(**best_params_xgb))
    ])

best_xgb_model.fit(X,y)

Pipeline(steps=[('prep',
                 Pipeline(steps=[('Binning',
                                  Binning(col_to_bin='Age',
                                          new_col_name='Age_bins',
                                          num_bins=3)),
                                 ('GroupMeanEncoder_BP',
                                  GroupMeanEncoder(agg_col='BP',
                                                   groupby_col='Age_bins',
                                                   new_col_name='X1')),
                                 ('GroupMeanEncoder_Cholesterol',
                                  GroupMeanEncoder(agg_col='Cholesterol',
                                                   groupby_col='Age_bins',
                                                   new_col_name='X2')),
                                 ('GroupMeanEncoder_HR',
                                  Grou...
                               gamma=1.4373368015867447, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0468112533159457, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=5, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=2634, n_jobs=None,
                               num_parallel_tree=None, ...))])

## Test data:

In [70]:
test_path = '/kaggle/input/competitions/playground-series-s6e2/test.csv'
test_df = pd.read_csv(test_path)
test_df.head()

# Removing redundant features
X_test = test_df.drop('id', axis=1)

In [71]:
y_pred_xgb = best_xgb_model.predict_proba(X_test)[:,1]

/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [72]:
submission = pd.DataFrame({
    'id': test_df['id'],
    target: y_pred_xgb
})

submission.to_csv('submission.csv', index=False)

The score increased from 0.95371 --> 0.95372 with the updated LeaderBoard rank of 533/3277. Concluding by stating that clustering did contribute towards better prediction making but didnt boost the score by much. 